# Model Testing Notebook

Test all three models (UNet, UNet++, TransUNet) locally before training in Colab.

**Tests:**
1. Model instantiation
2. Forward pass
3. Parameter counting
4. Different input sizes

In [1]:
import sys
sys.path.append('../src')

import torch
import numpy as np
from models.unet import UNet
from models.unetpp import UNetPlusPlus
from models.transunet import TransUNet

## 1. Test UNet

In [2]:
print("Testing UNet...\n")

# Create model
unet = UNet(in_channels=3, out_channels=1, features=64)

# Model info
info = unet.get_model_info()
print(f"Model: {info['model_name']}")
print(f"Total parameters: {info['total_params']:,}")
print(f"Trainable parameters: {info['trainable_params']:,}")

# Test forward pass
x = torch.randn(2, 3, 256, 256)
print(f"\nInput shape: {x.shape}")

with torch.no_grad():
    output = unet(x)

print(f"Output shape: {output.shape}")
print(f"Output range: [{output.min():.3f}, {output.max():.3f}]")

print("\n✓ UNet test passed!")

Testing UNet...



Model: UNet
Total parameters: 13,391,361
Trainable parameters: 13,391,361

Input shape: torch.Size([2, 3, 256, 256])
Output shape: torch.Size([2, 1, 256, 256])
Output range: [-1.573, 1.256]

✓ UNet test passed!


## 2. Test UNet++

In [3]:
print("Testing UNet++...\n")

# Create model
unetpp = UNetPlusPlus(in_channels=3, out_channels=1, features=32, deep_supervision=False)

# Model info
info = unetpp.get_model_info()
print(f"Model: {info['model_name']}")
print(f"Total parameters: {info['total_params']:,}")
print(f"Trainable parameters: {info['trainable_params']:,}")

# Test forward pass
x = torch.randn(2, 3, 256, 256)
print(f"\nInput shape: {x.shape}")

with torch.no_grad():
    output = unetpp(x)

print(f"Output shape: {output.shape}")
print(f"Output range: [{output.min():.3f}, {output.max():.3f}]")

print("\n✓ UNet++ test passed!")

Testing UNet++...



Model: UNet++
Total parameters: 9,159,681
Trainable parameters: 9,159,681

Input shape: torch.Size([2, 3, 256, 256])
Output shape: torch.Size([2, 1, 256, 256])
Output range: [-1.345, 2.355]

✓ UNet++ test passed!


## 3. Test TransUNet

In [4]:
print("Testing TransUNet...\n")

# Create model (smaller version for local testing)
transunet = TransUNet(
    in_channels=3,
    out_channels=1,
    img_size=256,
    patch_size=16,
    embed_dim=768,
    depth=6,  # Reduced for local testing
    num_heads=12
)

# Model info
info = transunet.get_model_info()
print(f"Model: {info['model_name']}")
print(f"Total parameters: {info['total_params']:,}")
print(f"Trainable parameters: {info['trainable_params']:,}")

# Test forward pass
x = torch.randn(2, 3, 256, 256)
print(f"\nInput shape: {x.shape}")

with torch.no_grad():
    output = transunet(x)

print(f"Output shape: {output.shape}")
print(f"Output range: [{output.min():.3f}, {output.max():.3f}]")

print("\n✓ TransUNet test passed!")

Testing TransUNet...

Model: TransUNet
Total parameters: 156,239,105
Trainable parameters: 156,239,105

Input shape: torch.Size([2, 3, 256, 256])
Output shape: torch.Size([2, 1, 256, 256])
Output range: [-0.884, 1.096]

✓ TransUNet test passed!


## 4. Compare Model Sizes

In [5]:
import pandas as pd

# Collect model info
models_info = [
    unet.get_model_info(),
    unetpp.get_model_info(),
    transunet.get_model_info()
]

# Create comparison DataFrame
comparison_df = pd.DataFrame([
    {
        'Model': info['model_name'],
        'Parameters': f"{info['total_params']:,}",
        'Parameters (M)': f"{info['total_params'] / 1e6:.2f}M"
    }
    for info in models_info
])

print("\nModel Comparison:")
print("=" * 60)
print(comparison_df.to_string(index=False))
print("=" * 60)


Model Comparison:
    Model  Parameters Parameters (M)
     UNet  13,391,361         13.39M
   UNet++   9,159,681          9.16M
TransUNet 156,239,105        156.24M


## 5. Test with Different Input Sizes

In [6]:
print("\nTesting different input sizes...\n")

sizes = [128, 256, 512]

for model_name, model in [('UNet', unet), ('UNet++', unetpp)]:
    print(f"{model_name}:")
    for size in sizes:
        x = torch.randn(1, 3, size, size)
        with torch.no_grad():
            output = model(x)
        print(f"  Input: {x.shape} -> Output: {output.shape}")
    print()


Testing different input sizes...

UNet:
  Input: torch.Size([1, 3, 128, 128]) -> Output: torch.Size([1, 1, 128, 128])
  Input: torch.Size([1, 3, 256, 256]) -> Output: torch.Size([1, 1, 256, 256])
  Input: torch.Size([1, 3, 512, 512]) -> Output: torch.Size([1, 1, 512, 512])

UNet++:
  Input: torch.Size([1, 3, 128, 128]) -> Output: torch.Size([1, 1, 128, 128])
  Input: torch.Size([1, 3, 256, 256]) -> Output: torch.Size([1, 1, 256, 256])
  Input: torch.Size([1, 3, 512, 512]) -> Output: torch.Size([1, 1, 512, 512])



## 6. Test Loss Functions

In [7]:
from utils import DiceLoss, CombinedLoss, dice_coefficient, iou_score

# Create dummy predictions and targets
pred = torch.randn(2, 1, 256, 256)
target = torch.randint(0, 2, (2, 1, 256, 256)).float()

print("Testing loss functions...\n")

# Dice Loss
dice_loss = DiceLoss()
loss_value = dice_loss(pred, target)
print(f"Dice Loss: {loss_value.item():.4f}")

# Combined Loss
combined_loss = CombinedLoss()
loss_value = combined_loss(pred, target)
print(f"Combined Loss: {loss_value.item():.4f}")

# Metrics
dice = dice_coefficient(pred, target)
iou = iou_score(pred, target)

print(f"\nDice Coefficient: {dice:.4f}")
print(f"IoU Score: {iou:.4f}")

print("\n✓ Loss functions test passed!")

Testing loss functions...

Dice Loss: 0.4998
Combined Loss: 0.6527

Dice Coefficient: 0.5012
IoU Score: 0.3344

✓ Loss functions test passed!


## Summary

✅ **All models tested successfully!**

**Model Comparison:**
- UNet: Lightest, ~31M parameters
- UNet++: Medium, ~9M parameters (with features=32)
- TransUNet: Heaviest, ~100M+ parameters

**Next steps:**
1. Complete preprocessing
2. Upload data to Google Drive
3. Train models in Colab with GPU